In [1]:
import numpy as np
import nltk
import json
import pickle
import re
import random
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from tensorflow.keras.optimizers import SGD
from nltk.stem import WordNetLemmatizer

In [2]:
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [3]:
tokenized_words=[]
classes = []
doc = []
ignoring_words = ['?', '!']
data_file = open('/content/intents.json').read()
intents = json.loads(data_file)

In [4]:
for intent in intents['intents']:
    for pattern in intent['patterns']:

        #tokenize each word
        w = nltk.word_tokenize(pattern)
        tokenized_words.extend(w)
        #add documents in the corpus
        doc.append((w, intent['tag']))

        # add to our classes list
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

In [5]:
lemmatizer = WordNetLemmatizer()
lemmatized_words = [lemmatizer.lemmatize(words.lower()) for words in tokenized_words if w not in ignoring_words] #lemmatization

In [6]:
lemmatized_words = sorted(list(set(lemmatized_words)))
classes = sorted(list(set(classes)))

In [7]:
pickle.dump(lemmatized_words,open('lem_words.pkl','wb'))
pickle.dump(classes,open('classes.pkl','wb'))

In [8]:
training_data = []

empty_array = [0] * len(classes)

for d in doc:

    bag_of_words = []

    pattern = d[0]

    pattern = [lemmatizer.lemmatize(word.lower()) for word in pattern]

    for w in lemmatized_words:

        bag_of_words.append(1) if w in pattern else bag_of_words.append(0)

    output_row = list(empty_array)

    output_row[classes.index(d[1])] = 1

    training_data.append([bag_of_words, output_row])

random.shuffle(training_data)

training = np.array(training_data)

train_x = list(training[:,0])

train_y = list(training[:,1])

<ipython-input-8-5eb568a5f5c9>:25: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  training = np.array(training_data)


In [10]:
bot_model = Sequential()
bot_model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
bot_model.add(Dropout(0.5))
bot_model.add(Dense(64, activation='relu'))
bot_model.add(Dropout(0.5))
bot_model.add(Dropout(0.25))
bot_model.add(Dense(len(train_y[0]), activation='softmax'))

In [15]:
from keras.optimizers import SGD
sgd = SGD(learning_rate=0.01, momentum=0.9, nesterov=True)
bot_model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

In [18]:
x_train = np.array(train_x)
y_train = np.array(train_y)
hist = bot_model.fit(x_train, y_train, epochs=200, batch_size=5, verbose=1)

Epoch 1/200
4/4 [==============================] - 1s 7ms/step - loss: 1.4131 - accuracy: 0.1579
Epoch 2/200
4/4 [==============================] - 0s 6ms/step - loss: 1.4170 - accuracy: 0.1053
Epoch 3/200
4/4 [==============================] - 0s 4ms/step - loss: 1.4335 - accuracy: 0.4211
Epoch 4/200
4/4 [==============================] - 0s 5ms/step - loss: 1.2977 - accuracy: 0.3684
Epoch 5/200
4/4 [==============================] - 0s 7ms/step - loss: 1.2602 - accuracy: 0.3684
Epoch 6/200
4/4 [==============================] - 0s 6ms/step - loss: 1.1614 - accuracy: 0.4737
Epoch 7/200
4/4 [==============================] - 0s 7ms/step - loss: 1.1458 - accuracy: 0.5789
Epoch 8/200
4/4 [==============================] - 0s 5ms/step - loss: 1.1206 - accuracy: 0.5789
Epoch 9/200
4/4 [==============================] - 0s 5ms/step - loss: 1.0113 - accuracy: 0.7895
Epoch 10/200
4/4 [==============================] - 0s 5ms/step - loss: 0.8912 - accuracy: 0.6842
Epoch 11/200
4/4 [===========

In [19]:
bot_model.save('chatbot_model.h5', hist)

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [20]:
from keras.models import load_model
intents_file = json.loads(open('intents.json').read())
lem_words = pickle.load(open('lem_words.pkl','rb'))
classes = pickle.load(open('classes.pkl','rb'))
bot_model = load_model('chatbot_model.h5')

In [21]:
def cleaning(text):
    words = nltk.word_tokenize(text)
    words = [lemmatizer.lemmatize(word.lower()) for word in words]
    return words

In [22]:
def bag_ow(text, words, show_details=True):
    sentence_words = cleaning(text)
    bag_of_words = [0]*len(words)
    for s in sentence_words:
        for i,w in enumerate(words):
            if w == s:
                bag_of_words[i] = 1
    return (np.array(bag_of_words))

In [26]:
def class_prediction(sentence, model):
    p = bag_ow(sentence, lem_words,show_details=False)
    result = bot_model.predict(np.array([p]))[0]
    ER_THRESHOLD = 0.30
    f_results = [[i,r] for i,r in enumerate(result) if r > ER_THRESHOLD]
    f_results.sort(key=lambda x: x[1], reverse=True)
    intent_prob_list = []
    for i in f_results:
        intent_prob_list.append({"intent": classes[i[0]], "probability": str(i[1])})
    return intent_prob_list

In [24]:
def getbotResponse(ints, intents):
    tag = ints[0]['intent']
    intents_list = intents['intents']
    for intent in intents_list:
        if(intent['tag']== tag):
            result = random.choice(intent['responses'])
            break
    return result
def bot_response(text):
    ints = class_prediction(text, bot_model)
    response = getbotResponse(ints, intents)
    return response

In [27]:
for i in range(3):
  text = input("You : ")
  print("Bot : ",bot_response(text))

You : hi
1/1 [==============================] - 0s 26ms/step
Bot :  Hi there, how can I help you?
You : what can you do?
1/1 [==============================] - 0s 33ms/step
Bot :  I can help you to book flight tickets easily
You : thanks
1/1 [==============================] - 0s 22ms/step
Bot :  Happy to help you
